# Welche Dateitypen gibt es?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [3]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [4]:
import pandas as pd

query ="MATCH (f:Git:File) RETURN f.relativePath as relativePath"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [5]:
df.head()

,relativePath
0,None
1,src/test/java/junit/samples/ListTest.java
2,src/main/java/org/junit/rules/ExpectedExceptio...
3,src/test/java/org/junit/rules/ExpectedExceptio...
4,src/test/java/org/junit/tests/running/methods/...


Der folgende Codeabschnitt extrahiert die verschiedenen Dateitypen entsprechend der Dateiendung und zählt deren Häufigkeit. Die Dateitypen werden in der Variable <font face="Courier">datatype</font> und deren Häufigkeit in der Variable <font face="Courier">frequency</font> gespeichert.

In [10]:

datatypes = df['relativePath'].str.rsplit('.', 1).str[1]
series = datatypes.value_counts()
datatype = list(series.index)
frequency = list(series)
print(datatype)
print(frequency)
# Erzeuge die Kategorie "andere", in der alle Dateitypen gesammelt werden, die weniger oder genau 20 mal auftauchen
andere = 0
for wert in frequency[:]:
    index = frequency.index(wert)
    if wert <= 20:
        andere += wert
        datatype.remove(datatype[index])
        frequency.remove(wert)
frequency.append(andere)
datatype.append("andere")
print(datatype)
print(frequency)



['java', 'html', 'class', 'gif', 'txt', 'jar', 'md', 'xml', 'htm', 'ant', 'GIF', 'css', 'sh', 'js', 'template', 'properties', 'zip', 'cvsignore', 'prefs', 'gitignore', 'yml', 'txt~', 'asc', 'gitattributes', 'pl', 'bashemtote/tags/recent_fail_ensurecommitted', 'cmd', 'svg', 'fml', 'bashemtote/tags/first_fail_ensurecommitted', 'rb', 'releaserc', 'arx', 'png', 'classpath', 'vcm_meta', 'launch', '0/javadoc/package-list', 'bashemtote/tags/-f', 'project']
[1383, 80, 41, 36, 21, 19, 17, 14, 11, 9, 8, 6, 6, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['java', 'html', 'class', 'gif', 'txt', 'andere']
[1383, 80, 41, 36, 21, 126]


## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).

## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).



In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

labels = datatype
values = frequency

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

In [12]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [18]:
import pygal 
pie_chart = pygal.Pie()
pie_chart.title = 'Dateitypen'
for einzelneDateitypen in datatype:
    index= datatype.index(einzelneDateitypen)
    #dazugehörige frequency herausbekommen
    anzahl=frequency[index]
    pie_chart.add(einzelneDateitypen, anzahl)
display(HTML(base_html.format(rendered_chart=pie_chart.render(is_unicode=True))))